In [144]:
import pandas as pd
import numpy as np 
from datetime import datetime, timedelta

In [145]:
np.random.seed(42)

In [146]:
num_orders = 10000
num_products = 100
num_machines = 20

In [147]:
order_data = {
    "order_id": range(1, num_orders + 1),
    "order_date": [datetime.now() - timedelta(days=int(np.random.randint(1, 15))) for _ in range(num_orders)],
    "product_id": np.random.randint(1, num_products + 1, num_orders),
    "quantity_ordered": np.random.randint(1, 10, num_orders),
}

In [148]:
df_orders = pd.DataFrame(order_data)

In [149]:
warehouse_data = {
    "product_id": range(1, num_products + 1),
    "product_name": [f"product_{i}" for i in range(1, num_products + 1)],
    "quantity_in_stock": np.random.randint(0, 20, num_products),
    "machine_no": [f"mac-{np.random.randint(1, num_machines+1)}" for _ in range(num_products)],
    "available_date": [datetime.now() + timedelta(days=int(np.random.randint(1, 8))) for _ in range(num_products)]
}

In [150]:
df_warehouse = pd.DataFrame(warehouse_data)

In [151]:
shipment_data = {
    "order_id": range(1, num_orders + 1),
    "expected_delivery_date": [datetime.now() + timedelta(days=int(np.random.randint(1, 10))) for _ in range(num_orders)]
}

In [152]:
df_shipments = pd.DataFrame(shipment_data)

In [153]:
df_orders.to_csv('orders.csv', index=False)
df_warehouse.to_csv('warehouse.csv', index=False)
df_shipments.to_csv('shipments.csv', index=False)

In [154]:
print("Orders DataFrame:")
df_orders.head()

Orders DataFrame:


,order_id,order_date,product_id,quantity_ordered
0,1,2024-06-20 17:20:53.963224,41,3
1,2,2024-06-23 17:20:53.963224,50,9
2,3,2024-06-14 17:20:53.963224,87,3
3,4,2024-06-16 17:20:53.963224,71,7
4,5,2024-06-19 17:20:53.963224,59,4


In [155]:
print("Warehouse DataFrame:")
df_warehouse.head()

Warehouse DataFrame:


,product_id,product_name,quantity_in_stock,machine_no,available_date
0,1,product_1,2,mac-17,2024-06-30 17:20:54.097312
1,2,product_2,14,mac-13,2024-06-28 17:20:54.097312
2,3,product_3,4,mac-6,2024-06-30 17:20:54.097312
3,4,product_4,2,mac-8,2024-07-04 17:20:54.097312
4,5,product_5,5,mac-6,2024-06-30 17:20:54.097312


In [156]:
print("Shipments DataFrame:")
df_shipments.head()

Shipments DataFrame:


,order_id,expected_delivery_date
0,1,2024-07-05 17:20:54.155528
1,2,2024-07-06 17:20:54.155528
2,3,2024-07-04 17:20:54.155528
3,4,2024-07-04 17:20:54.155528
4,5,2024-07-02 17:20:54.155528


In [157]:
import os

In [158]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from sklearn.metrics.pairwise import cosine_similarity

In [162]:
def check_availability(order_data, df_warehouse, df_shipments):

    product_id = order_data["product_id"]
    quantity_ordered = order_data["quantity_ordered"]

    warehouse_stock = df_warehouse[df_warehouse["product_id"] == product_id]["quantity_in_stock"].values[0]

    if quantity_ordered <= warehouse_stock:
        available_quantity = quantity_ordered
        delayed_quantity = 0
        expected_delivery_date = df_shipments[df_shipments["order_id"] == order_data["order_id"]][
            "expected_delivery_date"
        ].values[0]
        df_warehouse.loc[df_warehouse["product_id"] == product_id, "quantity_in_stock"] -= quantity_ordered

    else:
        available_quantity = warehouse_stock
        delayed_quantity = quantity_ordered - warehouse_stock
        df_warehouse.loc[df_warehouse["product_id"] == product_id, "quantity_in_stock"] = 0
        creation_time_per_product = 2  
        additional_creation_time = int(delayed_quantity * creation_time_per_product)
        available_date = df_warehouse[df_warehouse["product_id"] == product_id]["available_date"].values[0]
        available_date = pd.to_datetime(df_warehouse[df_warehouse["product_id"] == product_id]["available_date"].values[0])
        additional_seconds = additional_creation_time * 3600 
        expected_delivery_date = available_date + timedelta(seconds=additional_seconds)

    return available_quantity, delayed_quantity, expected_delivery_date

In [163]:
def handle_order_availability(order_data, df_orders, df_warehouse, df_shipments):
    llm = ChatGoogleGenerativeAI(api_key=os.getenv("GOOGLE_API_KEY"), model="models/gemini-pro", temperature=0.7)

    prompt_template = PromptTemplate(
        input_variables=["quantity_ordered", "product_id", "available_quantity", "delayed_quantity", "expected_delivery_date"],
        template="""
        You are a customer service representative for an e-commerce company. 
        The customer has placed an order for {quantity_ordered} units of product {product_id}. 
        Please check the availability of the product in the warehouse and inform the customer 
        about the expected delivery date. If there's insufficient stock, inform the customer 
        about the available quantity and the delayed delivery for the remaining units. 
        Available quantity: {available_quantity}, Delayed quantity: {delayed_quantity}, 
        Expected delivery date: {expected_delivery_date}.
        What is the appropriate response to the customer?
        """
    )
    
    available_quantity, delayed_quantity, expected_delivery_date = check_availability(order_data, df_warehouse, df_shipments)
        
    chain = LLMChain(
        llm=llm,
        prompt=prompt_template
    )

    response = chain.run({
        "quantity_ordered": order_data["quantity_ordered"],
        "product_id": order_data["product_id"],
        "available_quantity": available_quantity,
        "delayed_quantity": delayed_quantity,
        "expected_delivery_date": expected_delivery_date,
        # "context": context
    })

    return response

In [164]:
def format_response(response):
    return "\n".join(response.split(". "))

try:
    order_id = 5
    order_data = df_orders[df_orders["order_id"] == order_id].iloc[0].to_dict()
    
    response = handle_order_availability(order_data, df_orders, df_warehouse, df_shipments)
    formatted_response = format_response(response)
    print(f"Customer Response:\n{formatted_response}")

except Exception as e:
    print(f"An error occurred: {e}")

Customer Response:
"Thank you for your order for 4 units of product 59
I have checked the availability in our warehouse and we currently have 4 units in stock
Therefore, all 4 units will be dispatched today and you can expect delivery by 2024-07-02T17:20:54.155528000
Please let us know if you have any further questions."


In [165]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')


product_descriptions = df_warehouse.apply(lambda row: f"Product ID: {row['product_id']}, Name: {row['product_name']}, Stock: {row['quantity_in_stock']}, Machine No: {row['machine_no']}, Available Date: {row['available_date']}", axis=1).tolist()
product_embeddings = model.encode(product_descriptions)
np.save('product_embeddings.npy', product_embeddings)


c:\Users\Raghurama Reddy\Desktop\summer24\Bristlecone-ecommerce\venv1\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [166]:
import faiss

product_embeddings = np.load('product_embeddings.npy')
dimension_products = product_embeddings.shape[1]
products_index = faiss.IndexFlatL2(dimension_products)
products_index.add(product_embeddings)


In [167]:
def generate_query_embedding(query):
    return model.encode([query])[0]

In [168]:
def search_similar_products(query_embedding, top_k=5):
    distances, indices = products_index.search(np.array([query_embedding]), top_k)
    return indices[0]

In [169]:
def generate_llm_response(query):
    query_embedding = generate_query_embedding(query)

 

    similar_product_indices = search_similar_products(query_embedding)
    similar_products = df_warehouse.iloc[similar_product_indices]

    prompt = f"Customer asked: {query}\n\n"


    prompt += "Similar products found:\n"
    for index, row in similar_products.iterrows():
        prompt += f"Product ID: {row['product_id']}, Name: {row['product_name']}, Stock: {row['quantity_in_stock']}, Available Date: {row['available_date']}\n"

    llm = ChatGoogleGenerativeAI(api_key=os.getenv("GOOGLE_API_KEY"), model="models/gemini-pro", temperature=0.7)

    prompt_template = PromptTemplate(
        input_variables=["prompt"],
        template="{prompt}"
    )

    chain = LLMChain(
        llm=llm,
        prompt=prompt_template
    )

    response = chain.run({
        "prompt": prompt
    })

    return response

In [171]:
query1 = "Are all the products ordered by product id 1 in stock?"
response1 = generate_llm_response(query1)
print(response1)

query2 = "Can we meet a customer order for quantity 66 of product id 10 on June 30, 2024?"
response2 = generate_llm_response(query2)
print(response2)

query3 = "When is the earliest that we can deliver order id 1 to the customer?"
response3 = generate_llm_response(query3)
print(response3)


Yes, the product with ID 1 is in stock. The stock is 2.
No, we cannot meet the customer order for quantity 66 of product id 10 on June 30, 2024. The available stock for product id 10 is 4, and it will be available on June 29, 2024.
2024-06-30
